In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import json

import librosa

import torch
import torchaudio as ta
import timm

from tqdm import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [16]:
data_root = '/kaggle/input/birdclef-2022'
train_meta = pd.read_csv(os.path.join(data_root, 'train_metadata.csv'))
ebird_taxonomy = pd.read_csv(os.path.join(data_root, 'eBird_Taxonomy_v2021.csv'))

In [17]:
train_meta.loc[:, 'secondary_labels'] = train_meta.secondary_labels.apply(eval)
train_meta['target_raw'] = train_meta.secondary_labels + train_meta.primary_label.apply(lambda x: [x])

In [18]:
all_species = sorted(set(train_meta.target_raw.sum()))
species2id = {s: i for i, s in enumerate(all_species)}
id2species = {i: s for i, s in enumerate(all_species)}

train_meta['target'] = train_meta.target_raw.apply(lambda species: [int(s in species) for s in all_species])

In [19]:
def load_wav(fname, offset, duration):
#     fname = 'afrsil1/XC125458.ogg'
    fpath = os.path.join(data_root, 'train_audio', fname)
    wav, sr = librosa.load(fpath, sr=None, duration=duration)
    assert sr <= 32000, sr
    return wav, sr

In [20]:
# %%time
# duration = 30
# sample_rate = 32000

# wav, sr = load_wav('afrsil1/XC125458.ogg', 5, duration)
# to_pad = duration * sample_rate - wav.shape[0]

# if to_pad > 0:
#     wav = np.pad(wav, (0, to_pad))



### Torch Dataset

In [21]:
from torch.utils.data import Dataset, DataLoader

class BirdDataset(Dataset):
    def __init__(self, df):
        super().__init__()
        self.df = df
        
    def __getitem__(self, idx):
        duration = 30
        sample_rate = 32000
        
        fname = self.df.iloc[idx]['filename']
        # TODO: add random offset
        wav, sr = load_wav(fname, 0, duration)
        to_pad = duration * sample_rate - wav.shape[0]
        if to_pad > 0:
            wav = np.pad(wav, (0, to_pad))
            
        target = self.df.iloc[idx]['target']
        
        # TODO: add weighting
            
        wav = torch.tensor(wav)
        target = torch.tensor(target, dtype=float)
        return {
            'wav': wav,
            'target': target,
        }

    def __len__(self):
        return len(self.df)

### Model

In [22]:
class Net(torch.nn.Module):
    def __init__(self, backbone_path=None):
        super().__init__()
        self.audio2image = self._init_audio2image()
        self.backbone = self._init_backbone()
        self.load_backbone(backbone_path)
        self.head = self._init_head(self.backbone.feature_info[-1]['num_chs'])      
        self.loss = torch.nn.BCEWithLogitsLoss()
        
    def forward(self, wav_tensor, y=None):
        spectrogram = self.audio2image(wav_tensor)
        spectrogram = spectrogram.permute(0, 2, 1)
        spectrogram = spectrogram[:, None, :, :]
        x = self.backbone(spectrogram)
        logits = self.head(x)
        
        if y is not None:
            loss = self.loss(logits, y)
        else:
            loss = None

        return {'loss': loss, 'logits': logits.sigmoid()}

    
    @staticmethod
    def _init_audio2image():
        mel = ta.transforms.MelSpectrogram(
            sample_rate=32000,
            n_fft=2048,
            win_length=2048,
            hop_length=512,
            f_min=16,
            f_max=16386,
            pad=0,
            n_mels=256,
            power=2,
            normalized=False,
        )
        db_scale = ta.transforms.AmplitudeToDB(top_db=80.0)
        audio2image = torch.nn.Sequential(mel, db_scale)
        return audio2image
    
    @staticmethod
    def _init_backbone():
        backbone = "resnet18"
        pretrained = False
        pretrained_weights = None
        train = True
        val = False
        in_chans = 1

        backbone = timm.create_model(
            backbone,
            pretrained=pretrained,
            num_classes=0,
            global_pool="",
            in_chans=in_chans,
        )
        return backbone
    
    @staticmethod
    def _init_head(num_chs):
        head = torch.nn.Sequential(
            torch.nn.AdaptiveAvgPool2d(output_size=1),
            torch.nn.Flatten(),
            torch.nn.Linear(num_chs, len(all_species))
        )
        return head
    
    def load_backbone(self, weights_path=None):
        if weights_path:
            state_dict=torch.load(weights_path)
            conv1_weight = state_dict['conv1.weight']
            state_dict['conv1.weight'] = conv1_weight.sum(dim=1, keepdim=True)
            state_dict.pop('fc.bias')
            state_dict.pop('fc.weight')
            self.backbone.load_state_dict(state_dict)
        

### Train loop

In [23]:
with open('../input/timm-pretrained-resnet/index.json') as fin:
    timm_index = json.load(fin)
resnet_path = os.path.join('../input/timm-pretrained-resnet/resnet', timm_index['resnet']['resnet18'])

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [25]:
train_meta, val_meta = train_test_split(train_meta, test_size=0.2)

In [26]:
model = Net(resnet_path)
train_dataset = BirdDataset(train_meta)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=2,
    pin_memory=False,
    drop_last=True,
)

val_dataset = BirdDataset(val_meta)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=2,
    pin_memory=False,
    drop_last=False,
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = torch.optim.Adam(model.parameters())

In [27]:
def train_epoch(model, optimizer, dataloader, device):
    tqdm_dataloader = tqdm(dataloader)
    loss_list = []
    model.train()
    for batch in tqdm_dataloader:
        loss = model(batch['wav'].to(device), batch['target'].to(device))['loss']
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    return loss_list
    

def val_epoch(model, dataloader, device):
    tqdm_dataloader = tqdm(dataloader)
    loss_list = []
    model.eval()
    y_true = None
    y_pred = None
    
    for batch in tqdm_dataloader:
        logits = model(batch['wav'].to(device))['logits']
        batch_target = batch['target'].cpu().numpy()
        batch_pred = logits.cpu().numpy()
        
        if y_true is None:
            y_true = batch_target
            y_pred = batch_pred
        else:
            y_true = np.vstack((y_true, batch_target))
            y_pred = np.vstack((y_pred, batch_pred))
        
    return y_true, y_pred
    

In [31]:
def score_pred(y_true, y_pred, trsh, score_conf):
    score_dict = {}
    for score_f, score_kwargs, score_prefix in score_conf:
        score_dict.update({
            f'{score_prefix}-{t}': score_f(y_true, y_pred > t, **score_kwargs)
            for t in trsh
        })
    return score_dict


import sklearn.metrics

def comp_metric(y_true, y_pred, epsilon=1e-9):
    """ Function to calculate competition metric in an sklearn like fashion

    Args:
        y_true{array-like, sparse matrix} of shape (n_samples, n_outputs)
            - Ground truth (correct) target values.
        y_pred{array-like, sparse matrix} of shape (n_samples, n_outputs)
            - Estimated targets as returned by a classifier.
    Returns:
        The single calculated score representative of this competitions evaluation
    """

    # Get representative confusion matrices for each label
    mlbl_cms = sklearn.metrics.multilabel_confusion_matrix(y_true, y_pred)

    # Get two scores (TP and TN SCORES)
    tp_scores = np.array([
        mlbl_cm[1, 1]/(epsilon+mlbl_cm[:, 1].sum()) \
        for mlbl_cm in mlbl_cms
        ])
    tn_scores = np.array([
        mlbl_cm[0, 0]/(epsilon+mlbl_cm[:, 0].sum()) \
        for mlbl_cm in mlbl_cms
        ])

    # Get average
    tp_mean = tp_scores.mean()
    tn_mean = tn_scores.mean()

    return round((tp_mean+tn_mean)/2, 8)

In [32]:
score_conf = [
    [f1_score, {'average': 'macro'}, 'f1'],
    [comp_metric, {}, 'comp_metric']
]

In [33]:

epochs = 1
model.to(device)
for e in range(epochs):
    epoch_loss = train_epoch(model, optimizer, train_dataloader, device)
    print(f'{e} train loss:', f'{np.mean(epoch_loss):.3f}', sep='\t')
    with torch.no_grad():
        y_true, y_pred = val_epoch(model, val_dataloader, device)
    score_dict = score_pred(
        y_true, y_pred,
        score_conf=score_conf,
        trsh={0.3, 0.4, 0.5, 0.6}
    )
    torch.save(model, f'{e}_model.pt')
    print(f'{e} val scores:')
    print(*[
        f'\t{case}: {case_score}' 
        for case, case_score in score_dict.items()
    ], sep='\n')


100%|██████████| 47/47 [01:57<00:00,  2.50s/it]


TypeError: score_pred() missing 1 required positional argument: 'score_conf'

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


### Submit

In [ ]:
with open(os.path.join(data_root, 'scored_birds.json')) as fin:
    test_birds = json.load(fin)

In [ ]:
class TestDataset(Dataset):
    def __init__(self, test_folder):
        super().__init__()
        self.test_folder = test_folder
        self.fnames = [f for f in os.listdir(test_folder) if f.endswith('.ogg')]
        
    def __len__(self):
        return len(self.fnames)
    
    def __getitem__(self, idx):
        fpath = os.path.join(self.test_folder, self.fnames[idx])
        wav, sr = load_wav(fpath, 0, None)
        wav = torch.tensor(wav)
        assert (13 * 5 * sr) > len(wav) 
        wav = wav[:len(wav) // 12 * 12].reshape((12, len(wav) // 12))
        return wav

In [ ]:
test_dataset = TestDataset(os.path.join(data_root, 'test_soundscapes'))
test_dataloader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=2,
    drop_last=False,   
)

In [ ]:
pred_list = []
treshold = 0.1
model.eval()
with torch.no_grad():
    for i, batch in tqdm(enumerate(test_dataloader)):
        batch_size, part_count, part_size = batch.shape
        batch = batch.reshape(batch_size * part_count, part_size)
        pred = model(batch.to(device))['logits']
        pred = pred.cpu().numpy()
        pred = pred > treshold
        
        for j, chunk_pred in enumerate(pred):
            inbatch_number = j // part_count
            chunk_number = j % part_count + 1
            f_idx = i * batch_size + inbatch_number
            fname = test_dataset.fnames[f_idx]
            prefix = fname.split('.')[0]
            sufix = f'{5 * chunk_number}'
            
            pred_list.extend([{
                'row_id': '_'.join([prefix, b, sufix]),
                'target': chunk_pred[species2id[b]]
            } for b in test_birds])
pred_pd = pd.DataFrame(pred_list)

In [ ]:
pred_pd.to_csv("submission.csv", index=False)
pred_pd

In [ ]:
# def chunk_wav(wav, sr, window_size):
#     chunks_count = len(wav) // (window_size * sr)
#     chunk_size = window_size * sr
#     chunks = []
#     for chunk_idx in range(chunks_count):
#         left = chunk_idx * sr
#         right = min(left + chunk_size, len(wav))
#         chunks.append(wav[left:right])
#     chunk_tensor = torch.tensor(chunks)
#     return chunk_tensor

# file_list = os.listdir(os.path.join(data_root, 'test_soundscapes'))

# # This is where we will store our results
# treshold = 0.5
# pred = {'row_id': [], 'target': []}
# model.eval()
# with torch.no_grad():
#     # Process audio files and make predictions
#     for fname in file_list:
#         prefix = fname.split('.')[0]
#         # Complete file path
#         fpath = os.path.join(data_root, 'test_soundscapes', fname)
#         wav, sr = load_wav(fpath, 0, None)
#         chunk_tensor = chunk_wav(wav, sr, window_size=5)

#         # Open file with librosa and split signal into 5-second chunks
#         # sig, rate = librosa.load(path)
#         # ...

#         # Let's assume we have a list of 12 audio chunks (1min / 5s == 12 segments)
#         chunk_score = model(chunk_tensor.to(device))['logits'].cpu().numpy()

#         # Make prediction for each chunk
#         # Each scored bird gets a random value in our case
#         # since we don't actually have a model
#         for i, all_score in enumerate(chunk_score):        
#             chunk_end_time = (i + 1) * 5
#             for bird in test_birds:

#                 # This is our random prediction score for this bird
#                 bird_score = all_score[species2id[bird]]

#                 # Assemble the row_id which we need to do for each scored bird
#                 row_id = prefix + '_' + bird + '_' + str(chunk_end_time)

#                 # Put the result into our prediction dict and
#                 # apply a "confidence" threshold of 0.5
#                 pred['row_id'].append(row_id)
#                 pred['target'].append(True if bird_score > treshold else False)


# # Make a new data frame and look at some results        
# results = pd.DataFrame(pred, columns = ['row_id', 'target'])

# # Quick sanity check
# print(results.head()) 
    
# # Convert our results to csv
# results.to_csv("submission.csv", index=False)    

In [ ]:
# test_fnames = [f for f in os.listdir(f'{data_root}/test_soundscapes') if f.endswith('.ogg')]
# test_pd = []
# for fname in test_fnames:
#     fpath = os.path.join(data_root, 'test_soundscapes', fname)
#     wav, sr = librosa.load(fpath, sr=None)
#     prefix = fname.split('.')[0]
#     window_size = 5 * sr
#     for i, chunk in enumerate(wav[::window_size]):
#         end_time = (i + 1) * 5
#         samples = [{
#             'row_id': f'{prefix}_{b}_{end_time}',
#             'file_id': fname,
#             'bird': b,
#             'end_time': end_time
#         } for b in test_birds]
#         test_pd.extend(samples)
        
# test_pd = pd.DataFrame(test_pd)
# test_pd